In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hammadjavaid/vehicle-object-detection-dataset-5-classes")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/vehicle-object-detection-dataset-5-classes


In [2]:
import os

dataset_path = "/kaggle/input/vehicle-object-detection-dataset-5-classes"

# Check if the directory exists
if os.path.exists(dataset_path):
    print(f"Contents of '{dataset_path}':")
    for item in os.listdir(dataset_path):
        print(item)
else:
    print(f"Error: Directory '{dataset_path}' not found.")

Contents of '/kaggle/input/vehicle-object-detection-dataset-5-classes':
vehicles


In [3]:
import os

# First, let's check what's actually in the dataset path
print(f"Exploring dataset structure in: {dataset_path}\n")

# Function to explore directory structure
def explore_directory(path, max_depth=5, current_depth=0):
    if current_depth > max_depth:
        return
    
    try:
        items = os.listdir(path)
        indent = "  " * current_depth
        print(f"{indent}Contents of '{path}':")
        
        for item in items:
            item_path = os.path.join(path, item)
            if os.path.isdir(item_path):
                print(f"{indent}  📁 {item}/")
                if current_depth < max_depth:
                    explore_directory(item_path, max_depth, current_depth + 1)
            else:
                print(f"{indent}  📄 {item}")
    except Exception as e:
        print(f"{indent}Error accessing {path}: {e}")

explore_directory(dataset_path, max_depth=2)

# Now try to find the correct dataset subdirectory
possible_paths = [
    os.path.join(dataset_path, 'dataset'),
    dataset_path,  # Sometimes the dataset folder IS the root
]

dataset_sub_path = None
for path in possible_paths:
    if os.path.exists(path):
        # Check if this path has 'images' and 'labels' folders
        contents = os.listdir(path)
        if 'images' in contents and 'labels' in contents:
            dataset_sub_path = path
            print(f"\n✅ Found dataset directory: {dataset_sub_path}")
            break

if dataset_sub_path is None:
    # If still not found, search for images and labels folders
    for root, dirs, files in os.walk(dataset_path):
        if 'images' in dirs and 'labels' in dirs:
            dataset_sub_path = root
            print(f"\n✅ Found dataset directory: {dataset_sub_path}")
            break

if dataset_sub_path:
    print(f"\nContents of dataset directory:")
    for item in os.listdir(dataset_sub_path):
        print(f"  - {item}")
else:
    print("\n❌ Could not find dataset directory with 'images' and 'labels' folders")


Exploring dataset structure in: /kaggle/input/vehicle-object-detection-dataset-5-classes

Contents of '/kaggle/input/vehicle-object-detection-dataset-5-classes':
  📁 vehicles/
  Contents of '/kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles':
    📄 data.yaml
    📁 val/
    Contents of '/kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles/val':
      📁 labels/
      📁 images/
    📁 test/
    Contents of '/kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles/test':
      📁 labels/
      📁 images/
    📁 train/
    Contents of '/kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles/train':
      📁 labels/
      📁 images/

✅ Found dataset directory: /kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles/val

Contents of dataset directory:
  - labels
  - images


In [4]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.9 MB/s eta 0:00:00a 0:00:01


In [5]:
import ultralytics

print(f"Ultralytics version: {ultralytics.__version__}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics version: 8.4.14


In [6]:
!nvidia-smi

Fri Feb 20 06:46:12 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
import yaml
from pathlib import Path

# Dataset root (read-only)
DATASET_DIR = Path("/kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles")

# Writable directory
WORKING_DIR = Path("/kaggle/working")

# ---- Read existing data.yaml ----
original_yaml = DATASET_DIR / "data.yaml"

if not original_yaml.exists():
    raise FileNotFoundError("❌ data.yaml not found in dataset directory")

with open(original_yaml, "r") as f:
    original_data = yaml.safe_load(f)

# Extract class info
classes = original_data.get("names", [])
nc = original_data.get("nc", len(classes))

if not classes or nc == 0:
    raise ValueError("❌ No classes found in original data.yaml")

# ---- Create new YOLO config ----
yolo_data_config = {
    "path": str(DATASET_DIR.resolve()),
    "train": "train/images",
    "val": "val/images",
    "test": "test/images",
    "nc": nc,
    "names": classes
}

# Save new YAML
yolo_yaml_path = WORKING_DIR / "data_yolo.yaml"
with open(yolo_yaml_path, "w") as f:
    yaml.dump(yolo_data_config, f, sort_keys=False)

# Print confirmation
print(f"✅ data_yolo.yaml file saved to: {yolo_yaml_path}\n")
print("Content of data_yolo.yaml:")
print(yolo_yaml_path.read_text())


✅ data_yolo.yaml file saved to: /kaggle/working/data_yolo.yaml

Content of data_yolo.yaml:
path: /kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles
train: train/images
val: val/images
test: test/images
nc: 5
names:
- bus
- car
- pickup
- truck
- van



In [ ]:
from ultralytics import YOLO
import torch
import gc
import os
import shutil

# ─────────────────────────────────────────────
# CONFIG
# ─────────────────────────────────────────────
MODEL_SIZE  = 's'               # 's' = 11M params, safe for T4
EPOCHS      = 30                # Kaggle session limit
IMG_SIZE    = 800
BATCH       = 16
PROJECT     = '/kaggle/working'
RUN_NAME    = f'yolov8{MODEL_SIZE}_tuned'

# ── Resume from previous session? ────────────
# If continuing from a previous session:
#   1. Set RESUME = True
#   2. Set RESUME_WEIGHTS to your uploaded last.pt path
RESUME          = False
RESUME_WEIGHTS  = '/kaggle/input/yolov8m-checkpoint/last.pt'

# ─────────────────────────────────────────────
# GPU Setup
# ─────────────────────────────────────────────
torch.cuda.empty_cache()
gc.collect()

print(f"🚀 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# ─────────────────────────────────────────────
# Load Model
# Fresh start → yolov8s.pt
# Resuming   → your saved last.pt
# ─────────────────────────────────────────────
if RESUME and os.path.exists(RESUME_WEIGHTS):
    model = YOLO(RESUME_WEIGHTS)
    print(f"♻️  Resuming from: {RESUME_WEIGHTS}")
else:
    model = YOLO(f'yolov8{MODEL_SIZE}.pt')
    print(f"✅ Fresh start: YOLOv8{MODEL_SIZE}")

print(f"   Parameters: {sum(p.numel() for p in model.model.parameters()):,}")

# ─────────────────────────────────────────────
# Training
# ─────────────────────────────────────────────
print(f"\n🏋️ Training: {EPOCHS} epochs | imgsz={IMG_SIZE} | batch={BATCH} | resume={RESUME}\n")

results = model.train(
    data=str(yolo_yaml_path),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH,
    device=0,
    cache='disk',
    workers=2,
    resume=RESUME,              # ← continues from last epoch if True

    patience=10,                # Tuned for 30 epochs (was 15)
    save=True,
    save_period=5,
    project=PROJECT,
    name=RUN_NAME,
    exist_ok=True,
    pretrained=True,
    optimizer='AdamW',
    verbose=True,
    seed=42,
    deterministic=False,

    # ── Learning Rate (faster convergence for 30 epochs) ──
    lr0=0.01,                   # Higher than 0.001 — converges faster
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=2.0,          # Was 3.0 — save an epoch for real training
    warmup_momentum=0.8,

    # ── Scheduler ─────────────────────────────
    cos_lr=True,
    close_mosaic=5,             # Was 15 — with 30 epochs, turn off mosaic at ep 25

    # ── Loss Weights ──────────────────────────
    box=9.0,
    cls=0.5,
    dfl=1.5,

    # ── Speed / Memory ────────────────────────
    amp=True,
    fraction=1.0,
    rect=False,
    single_cls=False,
    profile=False,

    # ── Augmentation ──────────────────────────
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=5.0,
    translate=0.1,
    scale=0.6,
    shear=2.0,
    perspective=0.0001,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
    copy_paste=0.1,
)

print("\n✅ Training complete!")

# ─────────────────────────────────────────────
# IMPORTANT: Save weights before session ends
# Download these files from Kaggle output panel
# ─────────────────────────────────────────────
save_dir = f'{PROJECT}/saved_weights'
os.makedirs(save_dir, exist_ok=True)

best_src = f'{PROJECT}/{RUN_NAME}/weights/best.pt'
last_src = f'{PROJECT}/{RUN_NAME}/weights/last.pt'

shutil.copy(best_src, f'{save_dir}/best.pt')
shutil.copy(last_src, f'{save_dir}/last.pt')

print(f"\n💾 Weights saved to {save_dir}/")
print(f"   ⬇️  DOWNLOAD THESE BEFORE SESSION ENDS:")
print(f"   → {save_dir}/best.pt  (use for inference)")
print(f"   → {save_dir}/last.pt  (use to resume next session)")
print(f"\n   Next session: set RESUME=True and upload last.pt as a Kaggle dataset")

# ─────────────────────────────────────────────
# Cleanup
# ─────────────────────────────────────────────
del model
torch.cuda.empty_cache()
gc.collect()

# ─────────────────────────────────────────────
# Validation — Standard + TTA
# ─────────────────────────────────────────────
assert os.path.exists(best_src), f"Weights not found: {best_src}"
best_model = YOLO(best_src)
print(f"\n📂 Loaded best model for validation")

# Standard
print("\n🔍 Standard validation...")
val_results = best_model.val(data=str(yolo_yaml_path), imgsz=IMG_SIZE, device=0)
print("\n📊 Standard Results:")
print(f"   mAP50:     {val_results.box.map50:.4f}")
print(f"   mAP50-95:  {val_results.box.map:.4f}")
print(f"   Precision: {val_results.box.mp:.4f}")
print(f"   Recall:    {val_results.box.mr:.4f}")

# TTA (+~1% free boost)
print("\n🔍 TTA validation (augment=True)...")
tta_results = best_model.val(data=str(yolo_yaml_path), imgsz=IMG_SIZE, device=0, augment=True)
print("\n📊 TTA Results:")
print(f"   mAP50:     {tta_results.box.map50:.4f}")
print(f"   mAP50-95:  {tta_results.box.map:.4f}")
print(f"   Precision: {tta_results.box.mp:.4f}")
print(f"   Recall:    {tta_results.box.mr:.4f}")

# Summary
baseline = 0.8639
gain = tta_results.box.map50 - baseline
print(f"\n🎯 Gain over baseline ({baseline}): {gain:+.4f}")

if tta_results.box.map50 >= 0.90:
    print("   ✅ TARGET HIT — 90%+ mAP50 achieved!")
elif tta_results.box.map50 >= 0.88:
    print("   🔶 Almost there! Resume next session with last.pt to push past 90%")
else:
    print("   ⚠️  Resume next session OR switch to MODEL_SIZE='m' with BATCH=8")

🚀 CUDA available: True
   GPU: Tesla T4
   VRAM: 15.6 GB
✅ Fresh start: YOLOv8s
   Parameters: 11,166,560

🏋️ Training: 30 epochs | imgsz=800 | batch=16 | resume=False

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=9.0, cache=disk, cfg=None, classes=None, close_mosaic=5, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/data_yolo.yaml, degrees=5.0, deterministic=False, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=800, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8s.pt, m

In [ ]:
from ultralytics import YOLO
import torch
import gc

# Clear GPU memory
torch.cuda.empty_cache()
gc.collect()

# Load a pre-trained YOLOv8n model
model = YOLO('yolov8n.pt')


# Optimized training configuration for Kaggle
print("Initiating model training...")
results = model.train(
    data=str(yolo_yaml_path),
    epochs=15,
    imgsz=640,
    batch=16,  # Adjust based on GPU memory (try 8 or 32)
    device=0,  # Use GPU
    cache='disk',  # Use disk caching instead of RAM (Kaggle has limited RAM)
    workers=2,  # Reduce workers to save RAM
    patience=5,  # Early stopping
    save=True,
    save_period=5,  # Save checkpoint every 5 epochs
    project='/kaggle/working',  # Save to Kaggle working directory
    name='yolov8_training',
    exist_ok=True,
    pretrained=True,
    optimizer='AdamW',  # Often works better than SGD
    verbose=True,
    seed=42,
    deterministic=False,  # Faster training
    single_cls=False,
    rect=False,
    cos_lr=True,  # Cosine learning rate scheduler
    close_mosaic=10,  # Disable mosaic augmentation in last 10 epochs
    amp=True,  # Automatic Mixed Precision for faster training
    fraction=1.0,  # Use full dataset
    profile=False,  # Disable profiling to save memory
    # Augmentation parameters
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    shear=0.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.0,
)

print("Model training completed.")

# Clear memory after training
del model
torch.cuda.empty_cache()
gc.collect()

# Load the best model for inference
best_model = YOLO('/kaggle/working/yolov8_training/weights/best.pt')

In [ ]:
from ultralytics import YOLO
from pathlib import Path

# Paths
WORKING_DIR = Path("/kaggle/working")
best_model_path = WORKING_DIR / "yolov8_training/weights/best.pt"

# Load the trained model
print(f"📂 Loading model: {best_model_path}")
model = YOLO(str(best_model_path))

# Validate
print("\n🔍 Validating YOLOv8 model...")
metrics = model.val()

# Display all metrics
print(f"\n📊 Validation Results:")
print(f"   mAP50:     {metrics.box.map50:.4f}")
print(f"   mAP50-95:  {metrics.box.map:.4f}")
print(f"   Precision: {metrics.box.mp:.4f}")
print(f"   Recall:    {metrics.box.mr:.4f}")

In [ ]:
import yaml
from pathlib import Path

# Dataset root (read-only)
DATASET_DIR = Path("/kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles")

# Writable directory
WORKING_DIR = Path("/kaggle/working")

# ---- Read existing data.yaml ----
original_yaml = DATASET_DIR / "data.yaml"

if not original_yaml.exists():
    raise FileNotFoundError("❌ data.yaml not found in dataset directory")

with open(original_yaml, "r") as f:
    original_data = yaml.safe_load(f)

# Extract class info
classes = original_data.get("names", [])
nc = original_data.get("nc", len(classes))

if not classes or nc == 0:
    raise ValueError("❌ No classes found in original data.yaml")

# ---- Create new YOLO config ----
yolo_data_config = {
    "path": str(DATASET_DIR.resolve()),
    "train": "train/images",
    "val": "val/images",
    "test": "test/images",
    "nc": nc,
    "names": classes
}

# Save new YAML
yolo_yaml_path = WORKING_DIR / "data_yolo.yaml"
with open(yolo_yaml_path, "w") as f:
    yaml.dump(yolo_data_config, f, sort_keys=False)

# Print confirmation
print(f"✅ data_yolo.yaml file saved to: {yolo_yaml_path}\n")
print("Content of data_yolo.yaml:")
print(yolo_yaml_path.read_text())


In [ ]:
from ultralytics import YOLO
import torch
import gc

# Clear GPU memory
torch.cuda.empty_cache()
gc.collect()

# Load a pre-trained YOLOv10n model
model = YOLO('yolov10n.pt')

# Optimized training configuration for Kaggle
print("Initiating model training...")
results = model.train(
    data=str(yolo_yaml_path),
    epochs=15,
    imgsz=640,
    batch=16,  # Adjust: 8 (safer), 16 (balanced), 32 (if no OOM)
    device=0,  # GPU
    cache='disk',  # Critical for Kaggle: disk cache instead of RAM
    workers=2,  # Reduce RAM usage
    patience=5,  # Early stopping
    save=True,
    save_period=5,  # Save checkpoint every 5 epochs
    project='/kaggle/working',  # Kaggle persistent directory
    name='yolov10_training',
    exist_ok=True,
    pretrained=True,
    optimizer='AdamW',
    verbose=True,
    seed=42,
    deterministic=False,
    amp=True,  # Automatic Mixed Precision - faster training
    cos_lr=True,  # Cosine LR scheduler
    close_mosaic=10,  # Disable mosaic in last 10 epochs
    profile=False,  # Save memory
    # Augmentation parameters (augment=True is deprecated, use individual params)
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    shear=0.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.0,
    copy_paste=0.0,
)

print("Model training completed.")

# Clear memory
del model
torch.cuda.empty_cache()
gc.collect()

# Load best model
best_model = YOLO('/kaggle/working/yolov10_training/weights/best.pt')
print(f"Best model loaded from: /kaggle/working/yolov10_training/weights/best.pt")

In [ ]:
from ultralytics import YOLO
from pathlib import Path

# Paths
WORKING_DIR = Path("/kaggle/working")
best_model_path = WORKING_DIR / "yolov10_training/weights/best.pt"

# Load the trained model
print(f"📂 Loading model: {best_model_path}")
model = YOLO(str(best_model_path))

# Validate
print("\n🔍 Validating YOLOv10n model...")
metrics = model.val()

# Display all metrics
print(f"\n📊 Validation Results:")
print(f"   mAP50:     {metrics.box.map50:.4f}")
print(f"   mAP50-95:  {metrics.box.map:.4f}")
print(f"   Precision: {metrics.box.mp:.4f}")
print(f"   Recall:    {metrics.box.mr:.4f}")

In [ ]:
import yaml
from pathlib import Path

# Dataset root (read-only)
DATASET_DIR = Path("/kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles")
# Writable directory
WORKING_DIR = Path("/kaggle/working")

# ---- Read existing data.yaml ----
original_yaml = DATASET_DIR / "data.yaml"
if not original_yaml.exists():
    raise FileNotFoundError("❌ data.yaml not found in dataset directory")

with open(original_yaml, "r") as f:
    original_data = yaml.safe_load(f)

# Extract class info
classes = original_data.get("names", [])
nc = original_data.get("nc", len(classes))

if not classes or nc == 0:
    raise ValueError("❌ No classes found in original data.yaml")

# ---- Create new YOLO config ----
yolo_data_config = {
    "path": str(DATASET_DIR.resolve()),
    "train": "train/images",
    "val": "val/images",
    "test": "test/images",
    "nc": nc,
    "names": classes
}

# Save new YAML
yolo_yaml_path = WORKING_DIR / "data_yolo.yaml"
with open(yolo_yaml_path, "w") as f:
    yaml.dump(yolo_data_config, f, sort_keys=False)

# Print confirmation
print(f"✅ data_yolo.yaml file saved to: {yolo_yaml_path}\n")
print("Content of data_yolo.yaml:")
print(yolo_yaml_path.read_text())

In [ ]:
from ultralytics import YOLO
import torch
import gc

# Clear GPU memory
torch.cuda.empty_cache()
gc.collect()

# Load a pre-trained YOLOv11n model
model = YOLO('yolo11n.pt')

# Train the model
print("🚀 Initiating YOLOv11 model training...")
results = model.train(
    data=str(yolo_yaml_path), 
    epochs=20, 
    imgsz=640, 
    batch=16,  # Adjust: 8 (safer), 16 (balanced), 24-32 (if no OOM)
    device=0,  # Enable GPU - CRITICAL for Kaggle!
    cache='disk',  # Use disk cache instead of RAM (Kaggle has limited RAM)
    workers=2,  # Reduce RAM usage from data loading
    patience=5,  # Early stopping to save GPU hours
    save=True,
    save_period=5,  # Save checkpoint every 5 epochs (safety for timeouts)
    project='/kaggle/working/yolov11_training',  # Kaggle persistent directory
    name='vehicle_detection',
    exist_ok=True,
    pretrained=True,
    optimizer='AdamW',  # Often better than SGD
    verbose=True,
    seed=42,
    deterministic=False,  # Faster training
    amp=True,  # Automatic Mixed Precision - faster on T4/P100
    cos_lr=True,  # Cosine learning rate scheduler
    close_mosaic=10,  # Disable mosaic in last 10 epochs for better accuracy
    profile=False,  # Disable profiling to save memory
    plots=True,  # Generate training plots
    # Augmentation parameters (augment=True is deprecated)
    hsv_h=0.015,  # Hue augmentation
    hsv_s=0.7,    # Saturation augmentation
    hsv_v=0.4,    # Value augmentation
    degrees=0.0,  # Rotation (0-10 for vehicles)
    translate=0.1,  # Translation
    scale=0.5,    # Scale augmentation
    shear=0.0,    # Shear
    perspective=0.0,  # Perspective
    flipud=0.0,   # Vertical flip (usually 0 for vehicles)
    fliplr=0.5,   # Horizontal flip (0.5 for vehicles)
    mosaic=1.0,   # Mosaic augmentation
    mixup=0.0,    # Mixup augmentation
    copy_paste=0.0,  # Copy-paste augmentation
)
print("✅ YOLOv11 model training completed.")

# Clear memory after training
del model
torch.cuda.empty_cache()
gc.collect()

# Load the best model for validation/inference
print("📦 Loading best trained model...")
best_model = YOLO('/kaggle/working/yolov11_training/vehicle_detection/weights/best.pt')
print(f"✅ Best model loaded from: /kaggle/working/yolov11_training/vehicle_detection/weights/best.pt")

# Optional: Validate the model
print("📊 Validating model performance...")
metrics = best_model.val()
print(f"mAP50-95: {metrics.box.map}")
print(f"mAP50: {metrics.box.map50}")
print(f"mAP75: {metrics.box.map75}")

In [ ]:
from ultralytics import YOLO
from pathlib import Path

# Paths
WORKING_DIR = Path("/kaggle/working")
best_model_path = WORKING_DIR / "yolov11_training/vehicle_detection/weights/best.pt"

# Load the trained model
print(f"📂 Loading model: {best_model_path}")
model = YOLO(str(best_model_path))

# Validate
print("\n🔍 Validating YOLOv11 model...")  # Fixed comment
metrics = model.val()

# Display all metrics
print(f"\n📊 Validation Results:")
print(f"   mAP50:     {metrics.box.map50:.4f}")
print(f"   mAP50-95:  {metrics.box.map:.4f}")
print(f"   Precision: {metrics.box.mp:.4f}")
print(f"   Recall:    {metrics.box.mr:.4f}")